Import all necessary packages

In [ ]:
import pandas as pd ## For data manipulation
import numpy as np ## For data manipulation
import requests ## To retrive website data
from bs4 import BeautifulSoup ## Website scraping

Extracting the table from wikipedia

In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table), header=0)[0] ## Using header to define columns names

Data manipulation

In [ ]:
df = df[(df.Borough != 'Not assigned')] ## Eliminate Not assigned Borough
df.loc[df['Neighborhood'] == 'Not assigned','Neighborhood'] = df['Borough'] ## Replace Not assigned Neighborhood with Borough.

In [ ]:
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index() ## Concat neighborhood

In [ ]:
df = df.rename(columns = {'Postcode': 'PostalCode'})

In [ ]:
df.shape

In [ ]:
geo = pd.read_csv('Geospatial_Coordinates.csv')
geo

In [ ]:
dfgeo = df.merge(geo, left_on = 'PostalCode', right_on = 'Postal Code', how = 'left')[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
dfgeo

In [ ]:
dfto = dfgeo[dfgeo['Borough'].str.contains('Toronto')]
dfto

In [ ]:
from geopy.geocoders import Nominatim
import folium

In [ ]:
address = 'Toronto, ON'

# geolocator = Nominatim(user_agent="to_explorer")
# location = geolocator.geocode(address)
latitude = 43.6532 # location.latitude -- I got timeout when reran the geocode
longitude = -79.3832 # location.longitude -- I got timeout when reran the geocode

In [ ]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfto['Latitude'], dfto['Longitude'], dfto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
# set number of clusters
kclusters = 5

toronto_clustering = dfto.drop('Neighborhood', 1)
toronto_clustering = toronto_clustering.drop('Borough', 1)
toronto_clustering 

In [ ]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)